In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import torch
import torch_geometric
import Bio.PDB as PDB
import kmbio  # fork of biopython PDB with some changes in how the structure, chain, etc. classes are defined.
import numpy as np
import proteinsolver

from proteinsolver.models.model import *
from proteinsolver.datasets import *

# custom stuff
#import proteinsolver_utils
#import proteinsolver_datasets
np.random.seed(1)

In [ ]:
#from_dir = "/home/sebastian/masters/data/210916_TCRpMHCmodels/models/"
#to_dir = "/home/sebastian/masters/data/neat_data/tcrpmhc/"
#model_suffix = "model_TCR-pMHC.pdb"
#for subdir in os.listdir(from_dir):
#    subdir_id = subdir.split("_")[0]
#    new_name = f"tcrpmhc_{subdir_id}.pdb"
#    os.system(f"mv {from_dir}/{subdir}/{model_suffix} {to_dir}/{new_name}")
#    
#from_dir = "/home/sebastian/masters/data/embedding_verification/raw_filtered_models"
#to_dir = "/home/sebastian/masters/data/neat_data/pmhc/"
#model_suffix = "model_pMHC.pdb"
#for subdir in os.listdir(from_dir):
#    subdir_id = subdir.split("_")[0]
#    new_name = f"pmhc_{subdir_id}.pdb"
#    os.system(f"mv {from_dir}/{subdir}/{model_suffix} {to_dir}/{new_name}")
#    
#from_dir = "/home/sebastian/masters/data/embedding_verification/raw_filtered_models"
#to_dir = "/home/sebastian/masters/data/neat_data/p/"
#model_suffix = "model_p.pdb"
#for subdir in os.listdir(from_dir):
#    subdir_id = subdir.split("_")[0]
#    new_name = f"p_{subdir_id}.pdb"
#    os.system(f"mv {from_dir}/{subdir}/{model_suffix} {to_dir}/{new_name}")

In [ ]:
UNIQUE_ID = "191f05de"
BEST_STATE_FILES = {
    #
    "191f05de": "/home/sebastian/proteinsolver/data/e53-s1952148-d93703104.state"
}
state_file = BEST_STATE_FILES[UNIQUE_ID]


#test_file = "/home/sebastian/proteinsolver/notebooks/protein_demo/inputs/1n5uA03.pdb"
#test_id = "1n5uA03.pdb"

test_file = "/home/sebastian/masters/data/test/3hfm.pdb"
test_id = "3hfm"

def load_model_paths(data_dir, model="model_TCR-pMHC.pdb"):
    model_list = list()
    for subdir in os.listdir(data_dir):
        path = f"{data_dir}/{subdir}/{model}"
        model_list.append(path)
    return np.array(model_list)

infiles = load_model_paths("/home/sebastian/masters/data/210916_TCRpMHCmodels/models/")

In [ ]:
structure_all = kmbio.PDB.load(infiles[0])
structure_all = merge_chains(structure_all)
structure = kmbio.PDB.Structure(test_id, structure_all[0].extract('A'))

pdata = proteinsolver.utils.extract_seq_and_adj(structure, 'A', remove_hetatms=True)
data = proteinsolver.datasets.row_to_data(pdata)
data = proteinsolver.datasets.transform_edge_attr(data)
data.to(device)

In [5]:
import torch.nn.functional as F

In [6]:
F.softmax()

TypeError: softmax() missing 1 required positional argument: 'input'

In [ ]:
#train_loader = iter(torch_geometric.data.DataLoader(d_train, batch_size=batch_size))
#valid_loader = iter(torch_geometric.data.DataLoader(d_valid, batch_size=batch_size))
#d = next(train_loader)
#x = torch.ones_like(d.x)*d.x.max().item()
#out = gnn(x, d.edge_index, d.edge_attr)
#
#net.eval()
#with torch.no_grad():
#    y = net(out.T.unsqueeze(0))
#    y = F.softmax(y, dim=0)


In [ ]:
def gnn_to_fnn(data, hidden_size, gnn_instance):
        data = data.to(device)
        y = data.y.to(device)
        with torch.no_grad():
            out = gnn_instance(data.x, data.edge_index, data.edge_attr)
        
        batches = torch.unique(data.batch)
        sliced_outs = list()
        pool = nn.AdaptiveAvgPool1d(output_size=hidden_size)
        for batch_idx in batches:
            batch_slice = torch.nonzero(data.batch == batch_idx)
            chain_map = data.chain_map[batch_idx]
            out_sliced = out[batch_slice]
            out_sliced = out_sliced[chain_map == "P"]  # get peptide only
            out_sliced = pool(out_sliced.T)
            sliced_outs.append(out_sliced)
        batched_out = torch.cat(sliced_outs, dim=1)
        return out, y